In [54]:
import torch
import torchvision


In [55]:
def calc_acc(pred,labels):
  _,pred_max=torch.max(preds,1)
  accuracy=torch.sum(pred_max==labels.data,dtype=torch.float64)/len(preds)
  return accuracy

In [56]:

class MyModel(torch.nn.Module):
  def __init__(self):
    super().__init__()

    #fully connected(dense) layer definition

    #mnist images size=28*28=784
    #128=number of output nerouns
    self.fc1=torch.nn.Linear(784,128)
    #previous output layer=next input layer
    #mnist has 10 output
    self.fc2=torch.nn.Linear(128,10)

  #model structure in forward method
  def forward(self,x):
    #x.shape[0]=batch_size
    #784=28*28
    #instead of having a 2 dimensional 28*28 matrice , we have a vector of size 784
    x=x.reshape((x.shape[0],784))
    x=self.fc1(x)
    x=torch.relu(x)
    x=torch.dropout(x,0.2,train=True)
    x=self.fc2(x)
    x=torch.softmax(x,dim=1)
    return x

In [57]:
device=torch.device("cuda")

model=MyModel()
model=model.to(device)
model.train(True)



MyModel(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [58]:
#Hyper Parameters
batch_size=64
epochs=10
lr=0.001

In [59]:
#Data Preparing

transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0),(1))      
])

dataset=torchvision.datasets.MNIST("./datasets",train=True,download=True,transform=transform)
train_data_loader=torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [60]:
#Model Compile
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
loss_function=torch.nn.CrossEntropyLoss()


In [61]:
#Train

for epoch in range(epochs):
  train_loss=0.0
  train_acc=0.0
  for images,labels in train_data_loader:
    images=images.to(device)
    labels=labels.to(device)
    optimizer.zero_grad()

    #1-forwarding
    preds=model(images)
    

    #2-backwarding
    loss=loss_function(preds,labels)
    loss.backward()

    #3-update
    optimizer.step()

    train_loss+=loss
    train_acc+=calc_acc(preds,labels)



  total_loss=train_loss/len(train_data_loader)
  total_acc=train_acc/len(train_data_loader)

  print(f"Epoch:{epoch+1},Loss{total_loss},Acc:{total_acc}")    
    




Epoch:1,Loss1.6164474487304688,Acc:0.8704690831556503
Epoch:2,Loss1.538353443145752,Acc:0.9309534914712153
Epoch:3,Loss1.5219826698303223,Acc:0.9455956823027718
Epoch:4,Loss1.5114009380340576,Acc:0.9544576226012793
Epoch:5,Loss1.5048885345458984,Acc:0.960954157782516
Epoch:6,Loss1.4992691278457642,Acc:0.9657182835820896
Epoch:7,Loss1.4955576658248901,Acc:0.9692997068230277
Epoch:8,Loss1.4925240278244019,Acc:0.9718150319829424
Epoch:9,Loss1.4903918504714966,Acc:0.9734974680170575
Epoch:10,Loss1.488181710243225,Acc:0.9757129530916844


In [64]:
#save 
#model.state_dict()#to see the model weights
torch.save(model.state_dict(),"mnist.pth") #to save the model weights

In [69]:
#inference
import cv2
import numpy as np


#model.train(False)
model.eval()

#preprocess
img=cv2.imread('7.jpg')
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.resize(img,(28,28))
tensor=transform(img).unsqueeze(0).to(device)

#process
preds=model(tensor)

#postprocess
preds=preds.cpu().detach().numpy()
output=np.argmax(preds)
output


7